In [1]:
import pandas as pd

In [2]:
gender_swapped_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/test_swapped_with_ann.csv", index_col=0)

In [3]:
gender_swapped_df.head()

,original,swapped,annotations
0,Його редактором був поет-символіст Яків Савчен...,Її редакторкою була поетеса-символістка Яна Са...,"{'поет-символіст': 'JOB', 'Яків Савченко': 'PE..."
1,Міжпредметні паралелі . Маніфест футуристів ск...,Міжпредметні паралелі . Маніфест футуристок ск...,"{'поет': 'JOB', 'Філіппо Марінетті': 'PERS', '..."
2,Помітною була організація « Гарт » ( 1923 — 19...,Помітною була організація « Гарт » ( 1923 — 19...,"{'Гарт': 'ORG', 'поет': 'JOB', '1923 — 1925': ..."
3,Її очолив байкар і прозаїк Сергій Пилипенко .,Його очолила байкарка і прозаїка Сергія Пилипе...,"{'байкар': 'JOB', 'прозаїк': 'JOB', 'Сергій Пи..."
4,Назва символізує незнищенність і нездоланність...,Назва символізує незнищенність і нездоланність...,{'поет': 'JOB'}


## How to evaluate the gender-swapper?

<b>Qualitative</b>:
- manual erros analysis
- generated names are dictionary words of the needed gender
- generated feminitives are dictionary words 
- Check for the agreement accuracy (pymorhy lib to check gender to avoid “вона робив”) stanza
- Cross similarity (double gender-swapping)
- POS consistency (whether words changed their part of speech) stanza

<b>Quantitative</b>:
- does the n of tokens coincide
- BERTScore (BLEU, ROUGE, and METEOR) (measuring how similar the changed text is to the original text) medium_article
- Retrain the NER model on the gender-swapped dataset
- the variety of feminitive forms generated (Names)
- different LLMs results comparison

In [10]:
gender_swapped_df['swapped'][0]

'Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.'

## Check for the agreement accuracy (pymorhy lib to check gender to avoid “вона робив”) stanza

In [89]:
# !pip install stanza

In [8]:
import stanza

print("Downloading Ukrainian model...")
stanza.download('uk')

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


2025-01-14 23:12:54 INFO: Downloaded file to /Users/linndfors/stanza_resources/resources.json
2025-01-14 23:12:54 INFO: Downloading default packages for language: uk (Ukrainian) ...
2025-01-14 23:12:56 INFO: File exists: /Users/linndfors/stanza_resources/uk/default.zip
2025-01-14 23:13:00 INFO: Finished downloading models and saved to /Users/linndfors/stanza_resources


In [9]:
# uk_nlp = stanza.Pipeline('uk', processors='tokenize,lemma,pos,depparse', verbose=False, use_gpu=False)
uk_nlp = stanza.Pipeline('uk', verbose=False, use_gpu=False)

In [18]:
uk_doc = uk_nlp('Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.')
print(type(uk_doc))

<class 'stanza.models.common.doc.Document'>


In [68]:
for i, sent in enumerate(uk_doc.sentences):
    print("[Sentence {}]".format(i+1))
    for word in sent.words:
        # print(dir(word))
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:s}".format(\
              word.text, word.lemma, word.pos, word.head, word.feats))
    print("")

[Sentence 1]
Співвласницею	співвласниця	NOUN  	0	Animacy=Anim|Case=Ins|Gender=Fem|Number=Sing
цієї        	цей         	DET   	3	Case=Gen|Gender=Fem|Number=Sing|PronType=Dem
компанії    	компанія    	NOUN  	1	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
у           	у           	ADP   	6	Case=Acc
свою        	свій        	DET   	6	Case=Acc|Gender=Fem|Number=Sing|Poss=Yes|PronType=Prs|Reflex=Yes
чергу       	черга       	NOUN  	12	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
з           	з           	ADP   	8	Case=Ins
часткою     	частка      	NOUN  	12	Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing
в           	в           	ADP   	11	Case=Acc
75          	75          	NUM   	11	Case=Gen|NumType=Card|Uninflect=Yes
%           	%           	NOUN  	8	Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur|Uninflect=Yes
є           	бути        	VERB  	1	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
Ірина       	Ірина       	PROPN 	12	Animacy=Anim|Case=Nom|Gender=Fem|NameType=Giv|Nu

TypeError: unsupported format string passed to NoneType.__format__

In [15]:
print("Mention text\tType\tStart-End")
for ent in uk_doc.ents:
    print("{}\t{}\t{}-{}".format(ent.text, ent.type, ent.start_char, ent.end_char))

Mention text	Type	Start-End


In [66]:
def check_gender_agreement_in_sentences(sentences):
    for sentence in sentences:
        doc = uk_nlp(sentence)
        

        for sent in doc.sentences:
            for word in sent.words:
                try:
                    if word.deprel in ["nsubj", "nsubj:pass"] and "Gender" in str(word.feats):
                        if "Animacy=Inan" not in str(word.feats):
                            subj_text = word.text
                            subj_feats = word.feats
                            subj_gender = [feat.split('=')[1] for feat in subj_feats.split('|') if feat.startswith('Gender')][0]
                            subj_number = [feat.split('=')[1] for feat in subj_feats.split('|') if feat.startswith('Number')][0]

                            head_id = word.head
                            if head_id > 0:
                                verb = sent.words[head_id - 1]
                                if verb.pos == "VERB" and "Gender" in str(verb.feats):
                                    verb_text = verb.text
                                    verb_feats = verb.feats
                                    verb_gender = [feat.split('=')[1] for feat in verb_feats.split('|') if feat.startswith('Gender')][0]
                                    verb_number = [feat.split('=')[1] for feat in verb_feats.split('|') if feat.startswith('Number')][0]

                                    if subj_gender == verb_gender and subj_number == verb_number:
                                        pass
                                        # print(f"AGREE: {subj_text} ({subj_feats}) і {verb_text} ({verb_feats})")
                                    else:
                                        print("\nSentence:", sentence)
                                        print(f"XXX NOT AGREE XXX: {subj_text} ({subj_feats}) і {verb_text} ({verb_feats})")
                except Exception as e:
                    print(f"Error {e}; in the following sentence: {sentence}")

In [67]:
check_gender_agreement_in_sentences(list(gender_swapped_df['swapped']))


Sentence: Григорія Стерпул стала відомою як бізнес-партнерка президентки регіональної Торгово-промислової палати Миколаївської області Ігоря Катвалюка, який раніше був заступником голови Миколаївської облдержадміністрації.
XXX NOT AGREE XXX: Григорія (Animacy=Anim|Case=Nom|Gender=Masc|NameType=Giv|Number=Sing) і стала (Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin)

Sentence: Слідча повідомила, що до прокуратури звернулася з заявою голова Громадської ради при виконавчому комітеті Лиманської міськради.
XXX NOT AGREE XXX: голова (Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing) і звернулася (Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin)
Error list index out of range; in the following sentence: Обидві Колеснікови згадувались як членкині наглядової ради ЗАТ «Енергомережа» до того, як фірму очолила Марина Крючкова, що останні роки представляє інтереси нардепки від БПП Ігоря Кононенка.

Sentence: Мати голови правління ПАТ «Херсонська ТЕЦ» в кінці 2

## Tokens count

In [90]:
def token_count_match(row):
        tokens_1 = len(uk_nlp(row['original']).sentences[0].tokens)
        tokens_2 = len(uk_nlp(row['swapped']).sentences[0].tokens)
        
        if tokens_1 != tokens_2:
              print(row['original'])
              print(row['swapped'])

for i, row in gender_swapped_df.iterrows():
    token_count_match(row)

> «Це все виконується силами «Запоріжелектротрансу», що мені дуже приємно» – > підкреслив міський голова Володимир Буряк.
«Це все виконується силами «Запоріжелектротрансу», що мені дуже приємно» – > підкреслила міська голова Володимира Буряк.
Григорій Стерпул відомий як бізнес-партнер президента регіональної Торгово-промислової палати Миколаївської області Ігоря Катвалюка, який раніше був заступником голови Миколаївської облдержадміністрації.
Григорія Стерпул стала відомою як бізнес-партнерка президентки регіональної Торгово-промислової палати Миколаївської області Ігоря Катвалюка, який раніше був заступником голови Миколаївської облдержадміністрації.
Олена Добича, згідно з останніми даними SMIDA, володіє 10,22% акцій ПрАТ «Бердянський райагропромпостач», головою правління якого Олександр Пономарьов був до свого обрання народним депутатом.
Олександра Добич стала володільницею 10,22% акцій ПрАТ «Бердянський райагропромпостач», головою правління якого Олександра Пономарьова була до свого

## POS comparison

In [4]:
from tqdm import tqdm

In [15]:
def compare_pos_tags(df):
    # results = []

    for _, row in tqdm(df.iterrows()):
        sent1, sent2 = row['original'], row['swapped']

        doc1 = uk_nlp(sent1)
        doc2 = uk_nlp(sent2)
        error_flag = False

        tokens1 = [(word.text, word.upos) for sent in doc1.sentences for word in sent.words]
        tokens2 = [(word.text, word.upos) for sent in doc2.sentences for word in sent.words]

        for (tok1, pos1), (tok2, pos2) in zip(tokens1, tokens2):
            if tok1 != tok2:
                if pos1 != pos2:
                    error_flag = True
                    print("XX BAD XX", (tok1, pos1, tok2, pos2))
        if error_flag:
            print(sent1)
            print(sent2)

In [16]:
for_pos_check_gender_swapped_df = gender_swapped_df.copy()

In [17]:
len(for_pos_check_gender_swapped_df)

539

In [18]:
compare_pos_tags(for_pos_check_gender_swapped_df)

15it [00:37,  2.46s/it]

XX BAD XX ('кокетлива', 'NOUN', 'кокетливий', 'ADJ')
В часи , коли талановитий драматург досягнув піку слави й популярності , при королівському дворі в Англії користувалася увагою кокетлива й вишукана Емілія Бассано Ланьєр .
В часи, коли талановита драматургиня досягла піку слави й популярності, при королівському дворі в Англії користувалася увагою кокетливий й вишуканий Еміль Бассано Ланьєр.


37it [01:27,  4.11s/it]

XX BAD XX ('свіжому', 'ADJ', 'свіж', 'NOUN')
В наших умовах у таких « рольових коконах » продовжували дефілювати партійні ортодокси , лектори ЦК Компартії , які своєю поведінкою дуже нагадували героя Кіфера Сазерленда ( чи то герой « Меланхолії » через двадцять п’ять років наслідує їх ?.. ) , та ще офіцери КҐБ , які єдині тоді мусили бути щасливі , бо їм додалося праці : в кінці травня один такий приходив вербувати й мене ( цей сюжет я потім використала для схожого епізоду в біографії Дарини в « Музеї покинутих секретів » ) , і ми з ним близько трьох годин мужньо блукали вулицями « на свіжому повітрі » , дивуючи нечастих перехожих ( він , правда , запропонував піти до ресторану – на той час гуляти « на свіжому повітрі » вже не рекомендувалося цілком офіційно , – але на це я згодитись не могла : ресторан – то вже , як не крути , « початок угоди » , після якого відмовлятися від « співпраці » було б набагато складніше , і наковтатися трохи зайвих мікрорентґенів – яка вже різниця , кільком

45it [01:43,  2.05s/it]

...

502it [14:55,  2.17s/it]

XX BAD XX ('який', 'DET', ',', 'PUNCT')
XX BAD XX ('володіє', 'VERB', 'яка', 'DET')
XX BAD XX ('100', 'NUM', 'володіє', 'VERB')
XX BAD XX ('%', 'NOUN', '100', 'NUM')
XX BAD XX ('статутного', 'ADJ', '%', 'NOUN')
XX BAD XX ('капіталу', 'NOUN', 'статутного', 'ADJ')
XX BAD XX ('є', 'AUX', 'капіталу', 'NOUN')
XX BAD XX ('громадянин', 'NOUN', ',', 'PUNCT')
XX BAD XX ('України', 'PROPN', 'є', 'AUX')
XX BAD XX ('Ткачук', 'PROPN', 'громадянка', 'NOUN')
XX BAD XX (',', 'PUNCT', 'Ірина', 'PROPN')
XX BAD XX ('який', 'DET', 'Борисівна', 'PROPN')
XX BAD XX ('також', 'ADV', ',', 'PUNCT')
XX BAD XX ('є', 'AUX', 'яка', 'DET')
XX BAD XX ('одним', 'DET', 'також', 'ADV')
XX BAD XX ('з', 'ADP', 'є', 'AUX')
XX BAD XX ('засновників', 'NOUN', 'одною', 'DET')
XX BAD XX ('ТОВ', 'NOUN', 'з', 'ADP')
XX BAD XX ('«', 'PUNCT', 'засновниць', 'NOUN')
XX BAD XX ('Авіаресурс', 'PROPN', 'ТОВ', 'NOUN')
XX BAD XX (',', 'PUNCT', 'Авіаресурс', 'PROPN')
XX BAD XX ('та', 'CCONJ', '»', 'PUNCT')
XX BAD XX ('володіє', 'VERB', ','

513it [15:13,  1.88s/it]

XX BAD XX ('слідчий', 'NOUN', 'слідча', 'ADJ')
XX BAD XX ('наклала', 'VERB', 'суддя', 'NOUN')
XX BAD XX ('слідчий', 'ADJ', 'Наталій', 'PROPN')
XX BAD XX ('суддя', 'NOUN', 'Зубачик', 'PROPN')
XX BAD XX ('Наталія', 'PROPN', '.', 'PUNCT')
Нагадаємо, 19 серпня арешт на вказані кошти слідчий наклала слідчий суддя Наталія Зубачик.
Нагадаємо, 19 серпня арешт на вказані кошти слідча суддя Наталій Зубачик.


539it [15:53,  1.77s/it]


In [91]:
import os
import numpy as np
import spacy
import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [92]:
ner_nlp = spacy.load("uk_ner_web_trf_13class")

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import json

with open ("../fem_dictionary.json") as js:
    musc_dict = json.load(js)

fem_dict = dict(reversed(list(musc_dict.items())))

In [129]:
len(ner_results_swapped)

12

In [130]:
type(ner_results_swapped)

spacy.tokens.doc.Doc

In [ ]:
dict_samples = 0
not_dict_samples = 0
total_samples = 0

for x, row in gender_swapped_df.iterrows():
    ner_results_orig = ner_nlp(row['original'])
    ner_results_swapped = ner_nlp(row['swapped'])
    if len(ner_results_orig.ents) == len(ner_results_swapped.ents):
        for i, _ in enumerate(ner_results_orig.ents):
            try:
                if ner_results_orig.ents[i].label_ == "JOB" and ner_results_swapped.ents[i].label_ == "JOB":
                    # print("lemma:", ner_results_orig.ents[i].lemma_, "\n-------")
                    if " " not in ner_results_orig.ents[i].text:
                        total_samples += 1
                        orig_doc = uk_nlp(ner_results_orig.ents[i].text)
                        swapped_doc = uk_nlp(ner_results_swapped.ents[i].text)
                        
                        orig_nominative = " ".join([word.lemma for sent in orig_doc.sentences for word in sent.words])
                        swapped_nominative = " ".join([word.lemma for sent in swapped_doc.sentences for word in sent.words])
                        
                        # print("lemma:", orig_nominative, ",", swapped_nominative, "\n-------------")
                        if orig_nominative in musc_dict.keys():
                            if swapped_nominative != musc_dict[orig_nominative]:
                                print(f"relevant feminitive to the {orig_nominative} would be {musc_dict[orig_nominative]}, but it is {swapped_nominative}")
                                not_dict_samples += 1
                            else:
                                print(f"good example from dict: {orig_nominative} - {swapped_nominative}")
                                dict_samples += 1
                        elif orig_nominative in fem_dict.keys():
                            if swapped_nominative != fem_dict[orig_nominative]:
                                print(f"relevant feminitive to the {orig_nominative} would be {fem_dict[orig_nominative]}, but it is {swapped_nominative}")
                                not_dict_samples += 1
                            else:
                                print(f"GOOD example from dict: {orig_nominative} - {swapped_nominative}")
                                dict_samples += 1
                        else:
                            print(f"X this pair not from the dict: {orig_nominative} - {swapped_nominative}")
                            not_dict_samples += 1
                        # print(ner_results_orig.ents[i].label_ , ner_results_orig.ents[i].text)
                        # print(ner_results_swapped.ents[i].label_ , ner_results_swapped.ents[i].text)
                        print()
                    # job_entities_list.append(ent.text)
        # return job_entities_list
            except Exception as e:
                print(e)
                print()

good example from dict: депутат - депутатка

good example from dict: президент - президентка

X this pair not from the dict: комунальник - комунальниця

X this pair not from the dict: судді - судді

X this pair not from the dict: судді - судді

X this pair not from the dict: судді - судді

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: судді - судді

X this pair not from the dict: судді - судді

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: суддя - суддя

X this pair not from the dict: судді - судді

good example from dict: директор - директорка

good example from dict: чиновник - чиновниця

good example from dict: депутат - депутатка

X this pair not fro

In [134]:
print("all pairs of JOB entity:", total_samples)
print("percentage of dict pairs =", dict_samples/total_samples * 100, "%")
print("percentage of non dict pairs =", not_dict_samples/total_samples * 100, "%")

all pairs of JOB entity: 274
percentage of dict pairs = 45.25547445255474 %
percentage of non dict pairs = 54.74452554744526 %
